<a href="https://colab.research.google.com/github/minthammock/cap-stone/blob/dev/capstone_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow
import tensorflow.keras as keras

import pandas as pd

In [2]:
! git clone https://github.com/minthammock/cap-stone

Cloning into 'cap-stone'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 1), reused 17 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [25]:
dfTemp = pd.read_json('./cap-stone/politifact_info.json', orient='', )
df = pd.DataFrame()
for column in dfTemp.columns:
  df[column] = dfTemp[column][0]

In [29]:
display(df.head())
display(df.describe())

,author,quote,truth_value
0,Facebook posts,Says the new coronavirus vaccines contain toxi...,pants-fire
1,Bloggers,21% of people are having serious adverse event...,false
2,Ted Cruz,Says that guidance urging vaccinated people to...,pants-fire
3,Facebook posts,A 2018 executive order gives Trump the ability...,pants-fire
4,Facebook posts,President Donald Trump will hold his inaugurat...,pants-fire


,author,quote,truth_value
count,18436,18436,18436
unique,4263,18413,9
top,Facebook posts,On a cap-and-trade plan.,false
freq,1051,3,4240
